In [43]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split

from keras import Model , Sequential
from keras.layers import Dense

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go


In [10]:
def readData():
    #Hack to get path of Notebook (Data file is in same directory structure)
    
    if '_dh' in globals():
        current_folder = globals()['_dh'][0]
    else:
        script_path=__file__
        current_folder=dirname(script_path)

    # Calculating path to the test data
    data_location = os.path.join(current_folder,'..' ,'data' , 'data_gen.xlsx')
    data_location = os.path.normpath(data_location)

    wb = load_workbook(data_location, data_only=True)
    ws=wb['data']

    data={'x':[], 'y':[]}
    cur_row=2

    while ws['A'+str(cur_row)].value != None:
        data['x'].append(ws['A'+str(cur_row)].value)
        data['y'].append(ws['B'+str(cur_row)].value)
        cur_row += 1
    df = pd.DataFrame(data)
    train, test =  train_test_split(df, test_size=0.2)
    return train, test

In [110]:
def plotData(train, test, predictions):
    init_notebook_mode(connected=True)
    series1 = go.Scatter(
    x = train['x'],
    y = train['y'],
    name = 'Train',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgb(0,0,152)'
        )
    )

    series2 = go.Scatter(
        x = test['x'],
        y = test['y'],
        name = 'Test',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(0,152,0)'
        )
    )

    series3 = go.Scatter(
        x = test['x'],
        y = predictions.flatten(),
        name = 'Predictions',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(256,256,0)'
        )
    )
    
    data = [series1,  series3]
    
    layout = dict(title = 'Training , Test and predictions',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )
    
    fig = dict(data=data, layout=layout)
    
    iplot(fig)

In [ ]:
def defineModel():
    model = Sequential()
    model.add(Dense(15, input_dim=1, activation='sigmoid'))
    model.add(Dense(350, activation='sigmoid'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [111]:
train, test=readData()

train_x = np.array(train['x'], dtype=np.float32)
train_y = np.array(train['y'], dtype=np.float32)

test_x = np.array(test['x'], dtype=np.float32)
test_y = np.array(test['y'], dtype=np.float32)

In [116]:
model = defineModel()

In [118]:
model.fit(train_x,train_y, epochs=200, verbose=2, batch_size=32,validation_data=(test_x, test_y))

Train on 800 samples, validate on 201 samples
Epoch 1/200
 - 0s - loss: 10.6379 - val_loss: 6.9646
Epoch 2/200
 - 0s - loss: 38.8899 - val_loss: 77.6340
Epoch 3/200
 - 0s - loss: 50.4370 - val_loss: 8.7762
Epoch 4/200
 - 0s - loss: 21.0944 - val_loss: 12.7315
Epoch 5/200
 - 0s - loss: 36.5639 - val_loss: 7.0193
Epoch 6/200
 - 0s - loss: 10.1645 - val_loss: 7.6937
Epoch 7/200
 - 0s - loss: 13.5640 - val_loss: 6.2909
Epoch 8/200
 - 0s - loss: 19.9286 - val_loss: 7.0625
Epoch 9/200
 - 0s - loss: 20.7178 - val_loss: 12.6693
Epoch 10/200
 - 0s - loss: 18.5368 - val_loss: 6.0161
Epoch 11/200
 - 0s - loss: 12.7182 - val_loss: 7.7679
Epoch 12/200
 - 0s - loss: 9.8147 - val_loss: 18.2767
Epoch 13/200
 - 0s - loss: 11.2889 - val_loss: 8.2487
Epoch 14/200
 - 0s - loss: 10.6170 - val_loss: 5.9789
Epoch 15/200
 - 0s - loss: 9.5849 - val_loss: 8.0886
Epoch 16/200
 - 0s - loss: 13.5633 - val_loss: 13.4651
Epoch 17/200
 - 0s - loss: 11.4634 - val_loss: 5.3369
Epoch 18/200
 - 0s - loss: 8.0607 - val_lo

Epoch 152/200
 - 0s - loss: 8.5932 - val_loss: 1.9381
Epoch 153/200
 - 0s - loss: 4.3709 - val_loss: 2.5535
Epoch 154/200
 - 0s - loss: 3.4510 - val_loss: 2.0065
Epoch 155/200
 - 0s - loss: 3.4538 - val_loss: 1.9647
Epoch 156/200
 - 0s - loss: 8.3904 - val_loss: 5.3770
Epoch 157/200
 - 0s - loss: 4.0299 - val_loss: 2.1634
Epoch 158/200
 - 0s - loss: 4.5718 - val_loss: 2.5497
Epoch 159/200
 - 0s - loss: 3.3501 - val_loss: 1.7826
Epoch 160/200
 - 0s - loss: 3.8155 - val_loss: 13.8155
Epoch 161/200
 - 0s - loss: 5.1916 - val_loss: 1.9744
Epoch 162/200
 - 0s - loss: 3.6590 - val_loss: 3.0606
Epoch 163/200
 - 0s - loss: 15.1902 - val_loss: 4.3827
Epoch 164/200
 - 0s - loss: 7.2921 - val_loss: 13.6085
Epoch 165/200
 - 0s - loss: 19.5985 - val_loss: 36.3047
Epoch 166/200
 - 0s - loss: 26.7092 - val_loss: 27.0443
Epoch 167/200
 - 0s - loss: 9.2150 - val_loss: 17.5501
Epoch 168/200
 - 0s - loss: 130.9416 - val_loss: 31.4022
Epoch 169/200
 - 0s - loss: 34.1966 - val_loss: 7.8550
Epoch 170/200
 -

In [119]:
predictions = model.predict(test_x)

In [120]:
plotData(train, test, predictions)